# Notebook for the performance test report 
This Python code is used to generate a performance report PDF from PCM/uprof monitoring data collected during readout application tests.

The key functions are:
* plot_vars_comparison(): Plots performance metrics from PCM/uprof data for multiple tests into comparison plots. It generates a plot for each socket.
* create_report_performance(): Creates the full PDF report.
* Processes the raw PCM/uprof data if needed
    * Generates the comparison plots by calling plot_vars_comparison()
    * Adds intro text, table of tests, and the plots to the PDF
    * Prints CPU core pinning info for each test
    * It takes input data from a specified folder, processes it, generates plots in an output folder, and builds the PDF report with custom text, table, and plots.
    
Helper functions all bdefined in basic_functions.py:

* make_name_list(): Generates lists of file names in the input folder
* break_file_name(): Parses info from a file name
* add_new_time_format(): Adds a timestamp column to PCM/uprof data
* uprof_pcm_formatter(): Converts uprof data to PCM-like format
* json_info(): Prints CPU pinning info for a test

So in summary, it automates generating a performance report from raw monitoring data, including custom intro text, test info table, comparison plots, and configuration details.

In [ ]:
# Import the modules needed, defining paths and functions
from basic_functions import *

print('Cheking list of packages need it')
for package_i in list_py_package:
    debug_missing_module(module_name=package_i)

pcm_columns_list_0 = ['C0 Core C-state residency', 'Socket0 Memory Bandwidth',
                      'Socket0 Instructions Per Cycle', 'Socket0 Instructions Retired Any (Million)',
                      'Socket0 L2 Cache Misses', 'Socket0 L2 Cache Hits',
                      'Socket0 L3 Cache Misses', 'Socket0 L3 Cache Hits']
pcm_columns_list_1 = ['C0 Core C-state residency','Socket1 Memory Bandwidth',
                      'Socket1 Instructions Per Cycle', 'Socket1 Instructions Retired Any (Million)', 
                      'Socket1 L2 Cache Misses', 'Socket1 L2 Cache Hits',
                      'Socket1 L3 Cache Misses', 'Socket1 L3 Cache Hits']
uprof_columns_list_0 = [' Utilization (%) Socket0', 'Total Mem Bw (GB/s) Socket0', 
                        'IPC (Sys + User) Socket0', ' ', 
                        'L2 Miss (pti) Socket0', 'L2 Access (pti) Socket0',
                        'L3 Miss Socket0', 'L3 Miss % Socket0']
uprof_columns_list_1 = ['Utilization (%) Socket1', 'Total Mem Bw (GB/s) Socket1', 
                        'IPC (Sys + User) Socket1', ' ',
                        'L2 Miss (pti) Socket1', 'L2 Access (pti) Socket1',
                        'L3 Miss Socket1', 'L3 Miss % Socket1']
label_names = ['CPU Utilization (%)', 'Memory Bandwidth (GB/sec)', 
               'Instructions Per Cycle', 'Instructions Retired Any (Million)',
               'L2 Cache Misses (Million)', 'L2 Cache [Misses/Accesses] (%)',
               'L3 Cache Misses (Million)', 'L3 Cache [Misses/Accesses] (%)']
label_columns = ['Socket0', 'Socket1']

def plot_vars_comparison(input_dir, output_dir, pdf_name):
    X_plot, Y_plot_0, Y_plot_1, label_plot_0, label_plot_1 = [], [], [], [], []
    
    pcm_file, uprof_file, core_utilization_file, reformated_uprof_file, reformated_core_utilization_file, all_file, all_plots_file = make_name_list(input_dir)
    
    for i, file_i in enumerate(all_plots_file):    
        info = break_file_name(file_i)
        data_frame = pd.read_csv('{}/{}.csv'.format(input_dir, file_i))
        X_plot.append(data_frame['NewTime'].values.tolist())
                
        Y_tmp_0, Y_tmp_1, label_tmp_0, label_tmp_1 = [], [], [], []
        
        if info[0]=='grafana':
            for k, (columns_pcm_0, columns_pcm_1) in enumerate(zip(pcm_columns_list_0, pcm_columns_list_1)):
                Y_0, label_0 = get_column_val(data_frame, [columns_pcm_0], [label_columns[0]], file_i)  
                Y_1, label_1 = get_column_val(data_frame, [columns_pcm_1], [label_columns[1]], file_i)  
                Y_tmp_0.append(Y_0)
                label_tmp_0.append(label_0)
                Y_tmp_1.append(Y_1)
                label_tmp_1.append(label_1)
        else:
            for k, (columns_uprof_0, columns_uprof_1) in enumerate(zip(uprof_columns_list_0, uprof_columns_list_1)):
                Y_0, label_0 = get_column_val(data_frame, [columns_uprof_0], [label_columns[0]], file_i)
                Y_1, label_1 = get_column_val(data_frame, [columns_uprof_1], [label_columns[1]], file_i)
                Y_tmp_0.append(Y_0)
                label_tmp_0.append(label_0)
                Y_tmp_1.append(Y_1)
                label_tmp_1.append(label_1)
    
        Y_plot_0.append(Y_tmp_0)
        label_plot_0.append(label_tmp_0)
        Y_plot_1.append(Y_tmp_1)
        label_plot_1.append(label_tmp_1)
    
    # Here we make the plot:
    matplotlib.rcParams['font.family'] = 'DejaVu Serif'
    rows=cols=2
    rows_cols = rows*cols
    fig, axs = plt.subplots(rows, cols, figsize=(18, 8))
    plt.style.use('default')
    axs = axs.flatten()
    #axs[3].axis('off')
    
    for i in range(len(Y_plot_0)):  #number of files or tests
        for j in range(len(Y_plot_0[i])):  #number of metrix
            if j < rows_cols:
                label0_ij0 = re.sub('_', ' ', label_plot_0[i][j][0])
                axs[j].plot(X_plot[i], Y_plot_0[i][j][0], color=color_list[i], label=label0_ij0, linestyle=linestyle_list[0])
                axs[j].set_ylabel('{}'.format(label_names[j]))
                axs[j].set_xlabel('Time (min)')
                axs[j].grid(which='major', color='gray', linestyle='dashed')
                axs[j].legend(loc='upper left')
            else:
                pass
                
    plt.tight_layout()
    plt.savefig('{}/{}_results_{}_socket0.png'.format(output_dir, pdf_name, info[4]))
    print('{}/{}_results_{}_socket0.png'.format(output_dir, pdf_name, info[4]))
    plt.close() 
    
    fig, axs = plt.subplots(rows, cols, figsize=(18, 8))
    plt.style.use('default')
    axs = axs.flatten()   
    
    for i in range(len(Y_plot_0)):  
        for j in range(len(Y_plot_0[i])):
            if j < rows_cols:
                pass
            else:
                label0_ij0 = re.sub('_', ' ', label_plot_0[i][j][0])
                axs[j-rows_cols].plot(X_plot[i], Y_plot_0[i][j][0], color=color_list[i], label=label0_ij0, linestyle=linestyle_list[0])
                axs[j-rows_cols].set_ylabel('{}'.format(label_names[j]))
                axs[j-rows_cols].set_xlabel('Time (min)')
                axs[j-rows_cols].grid(which='major', color='gray', linestyle='dashed')
                axs[j-rows_cols].legend(loc='upper left')
                
    plt.tight_layout()
    plt.savefig('{}/{}_results_cache_{}_socket0.png'.format(output_dir, pdf_name, info[4]))
    print('{}/{}_results_cache_{}_socket0.png'.format(output_dir, pdf_name, info[4]))
    plt.close() 
    
    fig, axs = plt.subplots(rows, cols, figsize=(18, 8))
    plt.style.use('default')
    axs = axs.flatten()
    
    for i in range(len(Y_plot_1)):  
        for j in range(len(Y_plot_1[i])):
            if j < rows_cols:
                label1_ij0 = re.sub('_', ' ', label_plot_1[i][j][0])
                axs[j].plot(X_plot[i], Y_plot_1[i][j][0], color=color_list[i], label=label1_ij0, linestyle=linestyle_list[0])
                axs[j].set_ylabel('{}'.format(label_names[j]))
                axs[j].set_xlabel('Time (min)')
                axs[j].grid(which='major', color='gray', linestyle='dashed')
                axs[j].legend(loc='upper left')
            else:
                pass
    
    plt.tight_layout()
    plt.savefig('{}/{}_results_{}_socket1.png'.format(output_dir, pdf_name, info[4]))
    print('{}/{}_results_{}_socket1.png'.format(output_dir, pdf_name, info[4]))
    plt.close() 
    
    fig, axs = plt.subplots(rows, cols, figsize=(18, 8))
    plt.style.use('default')
    axs = axs.flatten()
    
    for i in range(len(Y_plot_1)):  
        for j in range(len(Y_plot_1[i])):
            if j < rows_cols:
                pass
            else:
                label1_ij0 = re.sub('_', ' ', label_plot_1[i][j][0])
                axs[j-rows_cols].plot(X_plot[i], Y_plot_1[i][j][0], color=color_list[i], label=label1_ij0, linestyle=linestyle_list[0])
                axs[j-rows_cols].set_ylabel('{}'.format(label_names[j]))
                axs[j-rows_cols].set_xlabel('Time (min)')
                axs[j-rows_cols].grid(which='major', color='gray', linestyle='dashed')
                axs[j-rows_cols].legend(loc='upper left')
    
    plt.tight_layout()
    plt.savefig('{}/{}_results_cache_{}_socket1.png'.format(output_dir, pdf_name, info[4]))
    print('{}/{}_results_cache_{}_socket1.png'.format(output_dir, pdf_name, info[4]))
    plt.close() 
    
def create_report_performance(input_dir, output_dir, daqconfs_cpupins_folder_parent_dir, process_pcm_files=False, process_uprof_files=False, print_info=True, streams='8, 16, 24, 32, 40, and 48', pdf_name='performance_report', repin_threads_file=None, comment=['TBA']):    
    directory([input_dir, output_dir])
    pcm_file, uprof_file, core_utilization_file, reformated_uprof_file, reformated_core_utilization_file, all_file, all_plots_file = make_name_list(input_dir)
    
    # Open pdf file
    pdf = FPDF()
    pdf.add_page()
    pdf.ln(1)
    pdf.image('{}/tools/dune_logo_3.jpg'.format(performancetest_path), w=180)
    pdf.ln(2)
    pdf.set_font('Times', 'B', 16)
    pdf.cell(40,10,'Performance Report')
    pdf.ln(10)
    
    # Processing the data first
    if process_pcm_files:
        for i, file_pcm_i in enumerate(pcm_file):
            add_new_time_format(input_dir, file_pcm_i)

    if process_uprof_files:
        for i, file_uprof_i in enumerate(uprof_file):
            uprof_pcm_formatter(input_dir, file_uprof_i)
            add_new_time_format(input_dir, 'reformatter_{}'.format(file_uprof_i))
    
    cpupins_utilazation_reformatter(input_dir)
    for i, file_core_i in enumerate(core_utilization_file):
         add_new_time_format_utilization(input_dir, 'reformatter_{}'.format(file_core_i))
        
    if process_pcm_files or process_uprof_files:
        print('Finish the processing of the data.')
    
    print(all_file[0])
    info_pcm_basic = break_file_name(all_file[0])
    
    # creating report
    pdf.set_font('Times', '', 10)
    pdf.write(5, 'The tests were run for the WIB{} data format for {} streams. The Figures 1 and 2 show the results of the tests ran (Table1) using the different metrics. \n'.format(info_pcm_basic[4], streams))
    pdf.write(5, '    * L2-hits is the fraction of requests that make it to L2 at all. Similar for L3. \n')
    pdf.write(5, '    * L2-misses is the fraction of requests that make it to L2 at all and then miss in L2. Similar for L3. \n')
    pdf.ln(10)
    
    #-------------------------------------------TABLE-----------------------------------------------
    # Data to tabular
    rows_data = []
    headers = ['Test', 'Readout SRV', 'dunedaq', 'OS', 'NODE', 'General comments']
    rows_data.append(headers)
    
    line_height = pdf.font_size * 2
    col_width = [pdf.epw/3.8, pdf.epw/8, pdf.epw/7, pdf.epw/12, pdf.epw/12, pdf.epw/5]  
    lh_list = [] #list with proper line_height for each row
    
    for i, file_i in enumerate(all_file):
        info = break_file_name(file_i)
        test_info = re.sub('_', ' ', info[5])
        line = [test_info, info[2], info[1], check_OS(info[2]), info[3], comment[i]]
        rows_data.append(line)
    
    # Determine line heights based on the number of words in each cell
    for row in rows_data:
        max_lines = 1  # Initialize with a minimum of 1 line
        for datum in row:
            lines_needed = len(str(datum).split('\n'))  # Count the number of lines
            max_lines = max(max_lines, lines_needed)
 
        lh_list.append(line_height * max_lines)
        
    # Add table rows with word wrapping and dynamic line heights
    for j, row in enumerate(rows_data):
        line_height_table = lh_list[j] 
        for k, datum in enumerate(row):
            pdf.multi_cell(col_width[k], line_height_table, datum, border=1, align='L', new_x=XPos.RIGHT, new_y=YPos.TOP, max_line_height=pdf.font_size)
            
        pdf.ln(line_height_table)
        
    pdf.write(5, 'Table 1. Summary of the tests ran. \n')    
    pdf.ln(10)
    
    #--------------------------------------------FIGURES------------------------------------------------
    plot_vars_comparison(input_dir, output_dir, pdf_name)
    
    if info[3] == '0' or info[3] == '01':
        pdf.image('{}/{}_results_{}_socket0.png'.format(output_dir, pdf_name, info_pcm_basic[4]), w=180)
        pdf.write(5, 'Figure 1. Socket0 results of the tests ran using the metrics CPU Utilization (%), Memory Bandwidth (GB/sec), Instructions Per Cycle, Instructions Retired Any (Million).')
        pdf.ln(10)
        pdf.image('{}/{}_results_cache_{}_socket0.png'.format(output_dir, pdf_name, info_pcm_basic[4]), w=180)
        pdf.write(5, 'Figure 2. Socket0 results of the tests ran using the metrics L2 Cache Misses (Million), L2 Cache [Misses/Hits] (%), L3 Cache Misses (Million), and L3 Cache [Misses/Hits] (%).')
        pdf.ln(10)
        
        if info[3] == '01':
            pdf.image('{}/{}_results_{}_socket1.png'.format(output_dir, pdf_name, info_pcm_basic[4]), w=180)
            pdf.write(5, 'Figure 3. Socket1 results of the tests ran using the metrics CPU Utilization (%), Memory Bandwidth (GB/sec), Instructions Per Cycle, Instructions Retired Any (Million).')
            pdf.ln(10)
            pdf.image('{}/{}_results_cache_{}_socket1.png'.format(output_dir, pdf_name, info_pcm_basic[4]), w=180)
            pdf.write(5, 'Figure 4. Socket1 results of the tests ran using the metrics L2 Cache Misses (Million), L2 Cache [Misses/Hits] (%), L3 Cache Misses (Million), and L3 Cache [Misses/Hits] (%).')
            pdf.ln(10)
        
    if info[3] == '1':
        pdf.image('{}/{}_results_{}_socket1.png'.format(output_dir, pdf_name, info_pcm_basic[4]), w=180)
        pdf.write(5, 'Figure 1. Socket1 results of the tests ran using the metrics CPU Utilization (%), Memory Bandwidth (GB/sec), Instructions Per Cycle, Instructions Retired Any (Million).')
        pdf.ln(10)
        pdf.image('{}/{}_results_cache_{}_socket1.png'.format(output_dir, pdf_name, info_pcm_basic[4]), w=180)
        pdf.write(5, 'Figure 2. Socket1 results of the tests ran using the metrics L2 Cache Misses (Million), L2 Cache [Misses/Hits] (%), L3 Cache Misses (Million), and L3 Cache [Misses/Hits] (%).')
        pdf.ln(10)
        
    #----------------------------------------CONFIGURATIONS---------------------------------------------
    pcm_file, uprof_file, core_utilization_file, reformated_uprof_file, reformated_core_utilization_file, all_file, all_plots_file = make_name_list(input_dir)
    
    if print_info:
        pdf.write(5, 'Configurations: \n', 'B')
        for i in range(len(all_file)):
            info = break_file_name(all_file[i])
            
            var_i='ru{}{}{}'.format(info[2], info[4], '0')
            file_daqconf_i='daqconf-{}-{}-{}-{}'.format(info[4], info[5], info[2], info[3])
            file_core_i='reformatter_core_utilization-{}-{}-{}-{}-{}'.format(info[1], info[2], info[3], info[4], info[5])
            
            if os.path.exists('{}/{}'.format(input_dir,file_core_i)):
                json_info(file_daqconf=file_daqconf_i, file_core=file_core_i, input_directory=daqconfs_cpupins_folder_parent_dir, input_dir=input_dir, var=var_i, pdf=pdf, if_pdf=print_info, repin_threads_file=repin_threads_file[i])
            else:
                print('Missing {}'.format(file_core_i))
                json_info(file_daqconf=file_daqconf_i, file_core='reformatter_core_utilization-all0', input_directory=daqconfs_cpupins_folder_parent_dir, input_dir='{}/tools'.format(daqconfs_cpupins_folder_parent_dir), var=var_i, pdf=pdf, if_pdf=print_info, repin_threads_file=repin_threads_file[i])
            pdf.cell(0, 10, 'Table {}. CPU core pins information for the "{}" test using dune_daq {}.'.format(i+2, info[5], info[1]))
            pdf.ln(10)           
            
    pdf.ln(20)
    pdf.set_font('Times', '', 10)
    pdf.write(5, 'The End, made on {}'.format(current_time()))
    pdf.output('{}/{}_report.pdf'.format(output_dir, pdf_name))
    
    print('The report was create and saved to {}/{}.pdf'.format(output_dir, pdf_name))
    
print('Ready to run and process')

## Proccesing data from Grafana
To extract the data from a given dashboard in grafana (Note: change the paths to fit yours): 
* extract_data_and_stats_from_panel(grafana_url, dashboard_uid, delta_time, host, partition, input_dir, output_csv_file)
* 'grafana_url' is:
    * 'http://np04-srv-009.cern.ch:3000'  (legacy)
    * 'http://np04-srv-017.cern.ch:31023' (new) 
* 'dashboard_uid' is the unic dashboard identifiyer, you can find this information on the link of the dashboard. The dashboard_uid code is in the web link after/d/.../ 
    * for intel-r-performance-counter-monitor-intel-r-pcm dashboard dashboard_uid = '91zWmJEVk'
    * for daq-overview dashboard dashboard_uid = 'v4_3_0-overview'
    * for frontend-ethernet dashboard dashboard_uid = 'v4_3_0-frontend_ethernet'
* delta_time is [start, end] given in the format '%Y-%m-%d %H:%M:%S'
* partition in some cases you will need to provide the partition name where the test was ran, for example "np04hddev"
* host is the name of the server in study, for example: "np02-srv-003"     
* output_csv_file (for performance tests): [version]-[server_app_tested]-[numa node]-[data format]-[tests_name]
    * example of name: v4_1_1-np02srv003-0-eth-stream_scaling

In [ ]:
# dvargas TPG adam tests

grafana_url = 'http://np04-srv-009.cern.ch:3000'
dashboard_uid = ['91zWmJEVk']
host_used = ['np04-srv-021','np04-srv-022','np04-srv-028','np04-srv-029']

delta_time_0 = [['2024-03-13 11:15:08', '2024-03-13 11:25:40'], 
                ['2024-03-13 10:47:52', '2024-03-13 10:55:30'], 
                ['2024-03-13 11:00:40', '2024-03-13 11:09:16']]

delta_time_4 = [['2024-03-13 10:47:52', '2024-03-13 10:55:30'], 
                ['2024-03-13 10:47:52', '2024-03-13 10:55:30'], 
                ['2024-03-13 10:47:52', '2024-03-13 10:55:30'], 
                ['2024-03-13 10:47:52', '2024-03-13 10:55:30']]
delta_time_5 = [['2024-03-13 11:15:08', '2024-03-13 11:25:40'], 
                ['2024-03-13 11:15:08', '2024-03-13 11:25:40'], 
                ['2024-03-13 11:15:08', '2024-03-13 11:25:40'], 
                ['2024-03-13 11:15:08', '2024-03-13 11:25:40']]
delta_time_6 = [['2024-03-13 11:00:40', '2024-03-13 11:09:16'], 
                ['2024-03-13 11:00:40', '2024-03-13 11:09:16'], 
                ['2024-03-13 11:00:40', '2024-03-13 11:09:16'], 
                ['2024-03-13 11:00:40', '2024-03-13 11:09:16']]

output_csv_file_0 = ['NFD_PROD4_240312-np04srv021-01-eth-SimpleThreshold', 
                     'NFD_PROD4_240312-np04srv021-01-eth-AbsRS', 
                     'NFD_PROD4_240312-np04srv021-01-eth-SilverBullet']
output_csv_file_1 = ['NFD_PROD4_240312-np04srv022-01-eth-SimpleThreshold', 
                     'NFD_PROD4_240312-np04srv022-01-eth-AbsRS', 
                     'NFD_PROD4_240312-np04srv022-01-eth-SilverBullet']
output_csv_file_2 = ['NFD_PROD4_240312-np04srv028-01-eth-SimpleThreshold', 
                     'NFD_PROD4_240312-np04srv028-01-eth-AbsRS', 
                     'NFD_PROD4_240312-np04srv028-01-eth-SilverBullet']
output_csv_file_3 = ['NFD_PROD4_240312-np04srv029-01-eth-SimpleThreshold', 
                     'NFD_PROD4_240312-np04srv029-01-eth-AbsRS', 
                     'NFD_PROD4_240312-np04srv029-01-eth-SilverBullet']

output_csv_file_4 = ['NFD_PROD4_240312-np04srv021-01-eth-AbsRS', 
                     'NFD_PROD4_240312-np04srv022-01-eth-AbsRS', 
                     'NFD_PROD4_240312-np04srv028-01-eth-AbsRS', 
                     'NFD_PROD4_240312-np04srv029-01-eth-AbsRS']
output_csv_file_5 = ['NFD_PROD4_240312-np04srv021-01-eth-SimpleThreshold', 
                     'NFD_PROD4_240312-np04srv022-01-eth-SimpleThreshold', 
                     'NFD_PROD4_240312-np04srv028-01-eth-SimpleThreshold', 
                     'NFD_PROD4_240312-np04srv029-01-eth-SimpleThreshold']
output_csv_file_6 = ['NFD_PROD4_240312-np04srv021-01-eth-SilverBullet', 
                     'NFD_PROD4_240312-np04srv022-01-eth-SilverBullet', 
                     'NFD_PROD4_240312-np04srv028-01-eth-SilverBullet', 
                     'NFD_PROD4_240312-np04srv029-01-eth-SilverBullet']

delta_time0 = [delta_time_0, delta_time_0, delta_time_0, delta_time_0]
output_csv_file0 = [output_csv_file_0, output_csv_file_1, output_csv_file_2, output_csv_file_3]
results_path0 = ['/eos/home-d/dvargas/SWAN_projects/performance_results/np04srv021_tpg',
                 '/eos/home-d/dvargas/SWAN_projects/performance_results/np04srv022_tpg',
                 '/eos/home-d/dvargas/SWAN_projects/performance_results/np04srv028_tpg',
                 '/eos/home-d/dvargas/SWAN_projects/performance_results/np04srv029_tpg']

delta_time1 = [delta_time_4, delta_time_5, delta_time_6]
output_csv_file1 = [output_csv_file_4, output_csv_file_5, output_csv_file_6]
results_path1 = ['/eos/home-d/dvargas/SWAN_projects/performance_results/tpg_AbsRS_test',
                 '/eos/home-d/dvargas/SWAN_projects/performance_results/tpg_SimpleThreshold_test',
                 '/eos/home-d/dvargas/SWAN_projects/performance_results/tpg_SilverBullet_test']

results_path_tpg = '/eos/home-d/dvargas/SWAN_projects/performance_results/adam_test_all'

for host_i, output_csv_file_i, results_path_i, delta_time_i in zip(host_used, output_csv_file0, results_path0, delta_time0):
    for output_csv_file_list, delta_time_list in zip(output_csv_file_i, delta_time_i):
        extract_data_and_stats_from_panel(grafana_url, dashboard_uid, delta_time=delta_time_list, 
                                          host=host_i, partition=None, input_dir=results_path_i, 
                                          output_csv_file=output_csv_file_list)

for output_csv_file_i, results_path_i, delta_time_i in zip(output_csv_file1, results_path1, delta_time1):
    for host_i, output_csv_file_list, delta_time_list in zip(host_used, output_csv_file_i, delta_time_i):
        extract_data_and_stats_from_panel(grafana_url, dashboard_uid, delta_time=delta_time_list, 
                                          host=host_i, partition=None, input_dir=results_path_i, 
                                          output_csv_file=output_csv_file_list)

for host_i, output_csv_file_i in zip(host_used, output_csv_file0):
    for output_csv_file_list, delta_time_list in zip(output_csv_file_i, delta_time_0):
        extract_data_and_stats_from_panel(grafana_url, dashboard_uid, delta_time=delta_time_list, 
                                          host=host_i, partition=None, input_dir=results_path_tpg, 
                                          output_csv_file=output_csv_file_list)

print('done :-)')

In [ ]:
# test adding RX Missed Errors → Frontend Ethernet

grafana_url = 'http://np04-srv-017.cern.ch:31023'
dashboard_uid = ['v4_3_0-frontend_ethernet', 'v4_3_0-overview']
partition = 'np04hddev'
host_used = 'np04-srv-022'
delta_time = [['2024-03-12 09:17:57', '2024-03-12 09:33:30'],
              ['2024-03-12 09:17:57', '2024-03-12 09:33:30']]
output_csv_file = ['v4_3_0-np04srv022-0-test0', 'v4_3_0-np04srv022-0-test1']
results_path = ['/eos/home-d/dvargas/SWAN_projects/performance_results/np04srv022']

for partition_list, delta_time_list, output_csv_file_list in zip(partition, delta_time, output_csv_file):
    extract_data_and_stats_from_panel(grafana_url, dashboard_uid, delta_time=delta_time_list, 
                                      host=host_used, partition=partition_list, input_dir=results_path, 
                                      output_csv_file=output_csv_file_list)

print('done :-)')

In [ ]:
# without raw recording

grafana_url = 'http://np04-srv-009.cern.ch:3000'
dashboard_uid = ['91zWmJEVk']
host_used = 'np04-srv-022'
partition = 'np04hddev'
delta_time = [['2024-03-14 20:17:57', '2024-03-14 21:33:30'],
              ['2024-03-14 16:24:47', '2024-03-14 16:40:36'],
              ['2024-03-14 10:07:52', '2024-03-14 10:23:44']]
output_csv_file = ['v4_3_0-np04srv022-0-k8s', 'v4_3_0-np04srv022-0-basic', 'v4_3_0-np04srv022-0-callbacks']
results_path = '/eos/home-d/dvargas/SWAN_projects/performance_results/np04srv022'

for partition_list, delta_time_list, output_csv_file_list in zip(partition, delta_time, output_csv_file):
    extract_data_and_stats_from_panel(grafana_url, dashboard_uid, delta_time=delta_time_list, 
                                      host=host_used, partition=None, input_dir=results_path, 
                                      output_csv_file=output_csv_file_list)

print('done :-)')

In [ ]:
# without raw recording

grafana_url = 'http://np04-srv-009.cern.ch:3000'
dashboard_uid = ['91zWmJEVk']
host_used = 'np02-srv-003'  
delta_time = [['2023-11-28 12:59:32', '2023-11-28 14:11:36'],
              ['2023-11-28 14:18:46', '2023-11-28 15:31:18']]
output_csv_file = ['v4_2_1-np02srv003-0-eth-stream_scaling', 
                   'v4_2_1-np02srv003-0-eth-stream_scaling_swtpg']
results_path = '/eos/home-d/dvargas/SWAN_projects/performance_results/without_recording'

for delta_time_list, output_csv_file_list in zip(delta_time, output_csv_file):
    extract_data_and_stats_from_panel(grafana_url, dashboard_uid, delta_time=delta_time_list, 
                                      host=host_used, partition=None, input_dir=results_path, 
                                      output_csv_file=output_csv_file_list)

print('done :-)')

In [ ]:
# with raw recording

grafana_url = 'http://np04-srv-009.cern.ch:3000'
dashboard_uid = ['91zWmJEVk']
host_used = 'np02-srv-003'  
delta_time = [['2023-11-28 15:38:44', '2023-11-28 16:50:50'],
              ['2023-11-29 09:59:53', '2023-11-29 11:12:27']]
output_csv_file = ['v4_2_0-np02srv003-0-eth-stream_scaling_recording', 
                   'v4_2_0-np02srv003-0-eth-stream_scaling_recording_swtpg']
results_path = '/eos/home-d/dvargas/SWAN_projects/performance_results/with_recording'

for delta_time_list, output_csv_file_list in zip(delta_time, output_csv_file):
    extract_data_and_stats_from_panel(grafana_url, dashboard_uid, delta_time=delta_time_list, 
                                      host=host_used, partition=None, input_dir=results_path, 
                                      output_csv_file=output_csv_file_list)

print('done :-)')

In [ ]:
# mman spsc queue test app

grafana_url = 'http://np04-srv-009.cern.ch:3000'
dashboard_uid = ['91zWmJEVk']
host_used = 'np02-srv-003'  
delta_time = [['2024-01-25 13:57:17', '2024-01-25 14:04:27']]
output_csv_file = ['v4_2_1-np02srv003-0-eth-queuetest_Intel']
results_path = '/eos/user/m/mman/performancetest/tools/queuetest_results'

for delta_time_list, output_csv_file_list in zip(delta_time, output_csv_file):
    extract_data_and_stats_from_panel(grafana_url, dashboard_uid, delta_time=delta_time_list, 
                                      host=host_used, partition=None, input_dir=results_path, 
                                      output_csv_file=output_csv_file_list)

print('done :-)')


## Performance report
To create the the performance report (Note: change the paths to fit yours): 
* create_report_performance(input_dir, output_dir, daqconfs_cpupins_folder_parent_dir, process_pcm_files=False, process_uprof_files=False, print_info=True, streams='8, 16, 24, 32, 40, and 48', pdf_name='performance_report', repin_threads_file=None, comment=['general comments about the run'])
    * 
    * 


In [ ]:
## dvargas ---> TPG standalone reports

results_path = '/eos/home-d/dvargas/SWAN_projects/performance_results/adam_test_all'
report_path = '/eos/home-d/dvargas/dunedaq_reports'
performancetest_path='/eos/home-m/mman/performancetest'
host_list = ['np04srv021','np04srv022','np04srv028','np04srv029']
comments_all=[' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ']
repin_threads_file='cpupin-absrs.json'

create_report_performance(input_dir=results_path, output_dir=report_path, 
                          daqconfs_cpupins_folder_parent_dir=performancetest_path, 
                          process_pcm_files=True, process_uprof_files=True, 
                          print_info=False, streams='48', 
                          pdf_name='tpg_AbsRS_SilverBullet_SimpleThreshold', comment=comments_all,
                          repin_threads_file=repin_threads_file)

paths0 = ['/eos/home-d/dvargas/SWAN_projects/performance_results/np04srv021_tpg',
         '/eos/home-d/dvargas/SWAN_projects/performance_results/np04srv022_tpg',
         '/eos/home-d/dvargas/SWAN_projects/performance_results/np04srv028_tpg',
         '/eos/home-d/dvargas/SWAN_projects/performance_results/np04srv029_tpg',
         '/eos/home-d/dvargas/SWAN_projects/performance_results/tpg_AbsRS_test']
names0 = ['np04srv021_tpg', 'np04srv022_tpg', 'np04srv028_tpg', 'np04srv029_tpg']
comments0 = [['TBA', 'TBA', 'TBA'], ['TBA', 'TBA', 'TBA'], ['TBA', 'TBA', 'TBA'], ['TBA', 'TBA', 'TBA']]

for list_path, list_name, list_comm in zip(paths0, names0, comments0):    
    create_report_performance(input_dir=list_path, output_dir=report_path, 
                              daqconfs_cpupins_folder_parent_dir=performancetest_path, 
                              process_pcm_files=True, process_uprof_files=True, 
                              print_info=False, streams='48', pdf_name=list_name, 
                              comment=list_comm, repin_threads_file=repin_threads_file)

paths1 = ['/eos/home-d/dvargas/SWAN_projects/performance_results/tpg_AbsRS_test',
          '/eos/home-d/dvargas/SWAN_projects/performance_results/tpg_SimpleThreshold_test',
          '/eos/home-d/dvargas/SWAN_projects/performance_results/tpg_SilverBullet_test']
names1 = ['tpg_AbsRS_test', 'tpg_SimpleThreshold_test', 'tpg_SilverBullet_test', ]
comments1 = [['TBA', 'TBA', 'TBA', 'TBA'], ['TBA', 'TBA', 'TBA', 'TBA'], ['TBA', 'TBA', 'TBA', 'TBA']]

for list_path, list_name, list_comm in zip(paths1, names1, comments1):    
    create_report_performance(input_dir=list_path, output_dir=report_path, 
                              daqconfs_cpupins_folder_parent_dir=performancetest_path, 
                              process_pcm_files=True, process_uprof_files=True, 
                              print_info=False, streams='48', pdf_name=list_name, 
                              comment=list_comm, repin_threads_file=repin_threads_file)

print('THE END')

In [ ]:
## dvargas

results_path_1 = '/eos/home-d/dvargas/SWAN_projects/performance_results/without_recording'
results_path_2 = '/eos/home-d/dvargas/SWAN_projects/performance_results/with_recording'
results_path_3 = '/eos/home-d/dvargas/SWAN_projects/performance_results/basic_stream_scaling_AMD'
report_path = '/eos/home-d/dvargas/dunedaq_reports'
performancetest_path = '/eos/home-d/dvargas/SWAN_projects/performancetest'

paths = [results_path_1, results_path_2, results_path_3]
names = ['performancetest_without_recording_np02srv004', 'performancetest_with_recording_np02srv004', 'performancetest_basic_stream_scaling_np02srv004_kernel_params']
comments = [['TBA', 'TBA'], ['TBA', 'TBA'], ['Before new kernel params', 'After new kernel params']]

for list_path, list_name, list_comm in zip(paths, names, comments):
    create_report_performance(input_dir=list_path, output_dir=report_path, 
                              daqconfs_cpupins_folder_parent_dir=performancetest_path, 
                              process_pcm_files=False, process_uprof_files=False, 
                              print_info=True, streams='8, 16, 24, 32, 40, and 48', 
                              pdf_name=list_name, comment=list_comm)

print('THE END')

In [ ]:
## mman readout queue perftest
results_path = '/eos/user/m/mman/performancetest/tools/queuetest_results'
report_path = '/eos/home-d/dvargas/dunedaq_reports'
performancetest_path = '/eos/user/m/mman/performancetest'

create_report_performance(input_dir=results_path, output_dir=report_path, 
                          daqconfs_cpupins_folder_parent_dir=performancetest_path, 
                          process_pcm_files=True, process_uprof_files=True, 
                          print_info=True, streams='4', 
                          pdf_name='queuetest_AMD_debug',
                          comment=['Unable to push within timeout period (timeout period was 0 milliseconds)', 
                                    'Unable to push within timeout period (timeout period was 0 milliseconds)'])

print('THE END')